<a href="https://colab.research.google.com/github/PG-1012/Cancer_Prediction_Model_Project/blob/main/ITM740_Asignment2_PG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import confusion_matrix


In [ ]:
cancer_data = pd.read_csv('/content/drive/MyDrive/ITM740/Assignment 2 /bcancer_data.csv')
cancer_data

,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
0,1,1,1,1,2,1,2,1,1,0
1,5,1,1,1,1,1,3,1,1,0
2,4,1,1,1,2,1,1,1,1,0
3,2,1,1,1,2,1,2,1,1,0
4,1,2,3,1,2,1,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...
535,1,1,1,1,2,1,1,1,1,0
536,3,1,1,1,2,1,2,3,1,0
537,4,1,1,1,2,1,1,1,1,0
538,2,1,1,1,2,1,1,1,1,0


In [ ]:
X = cancer_data.drop(columns=['Class']).values
y = cancer_data['Class'].values



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

(432, 9)

In [ ]:
X_train = torch.tensor(X_train).type(torch.float)
X_test = torch.tensor(X_test).type(torch.float)
y_train = torch.tensor(y_train).type(torch.float)
y_test = torch.tensor(y_test).type(torch.float)

In [ ]:
class Model_Version1(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear( in_features= 9, out_features= 10)
    self.layer_2 = nn.Linear( in_features= 10 , out_features= 1)
    self.relu = nn.ReLU()

  def forward(self, x):
      # Intersperse the ReLU activation function between layers
       return self.layer_2(self.relu(self.layer_1(x)))



In [ ]:
  model_print = Model_Version1()
  model_print

Model_Version1(
  (layer_1): Linear(in_features=9, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=1, bias=True)
  (relu): ReLU()
)

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model_print.parameters(), lr=0.1)

In [ ]:
torch.manual_seed(42)
epochs = 100

X_train, y_train = X_train, y_train
X_test, y_test = X_test, y_test

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
for epoch in range (epochs):
  y_logits = model_print(X_train).squeeze()
  y_pred = torch.round(torch.sigmoid(y_logits))

  log_loss = loss_fn(y_logits, y_train)
  acc = accuracy_fn(y_true=y_train, y_pred=y_pred)


  optimizer.zero_grad()

  log_loss.backward()

  optimizer.step()
  model_print.eval()
  with torch.inference_mode():
    test_logits = model_print(X_test).squeeze()
    test_pred = torch.round(torch.sigmoid(test_logits))
    test_loss = loss_fn(test_logits, y_test)
    test_acc = accuracy_fn(y_true=y_test,
                            y_pred=test_pred)

  # Print out what's happening
  if epoch % 10 == 0:
      print(f"Epoch: {epoch} | Loss: {log_loss:.5f}, Accuracy: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}%")


Epoch: 0 | Loss: 0.59503, Accuracy: 45.83% | Test Loss: 0.60051, Test Accuracy: 43.52%
Epoch: 10 | Loss: 0.47518, Accuracy: 86.81% | Test Loss: 0.47363, Test Accuracy: 89.81%
Epoch: 20 | Loss: 0.39977, Accuracy: 90.97% | Test Loss: 0.39224, Test Accuracy: 90.74%
Epoch: 30 | Loss: 0.34239, Accuracy: 92.82% | Test Loss: 0.32871, Test Accuracy: 93.52%
Epoch: 40 | Loss: 0.30094, Accuracy: 93.52% | Test Loss: 0.28188, Test Accuracy: 95.37%
Epoch: 50 | Loss: 0.27041, Accuracy: 94.68% | Test Loss: 0.24728, Test Accuracy: 95.37%
Epoch: 60 | Loss: 0.24758, Accuracy: 94.44% | Test Loss: 0.22208, Test Accuracy: 95.37%
Epoch: 70 | Loss: 0.23030, Accuracy: 94.68% | Test Loss: 0.20284, Test Accuracy: 97.22%
Epoch: 80 | Loss: 0.21654, Accuracy: 94.68% | Test Loss: 0.18714, Test Accuracy: 96.30%
Epoch: 90 | Loss: 0.20500, Accuracy: 94.68% | Test Loss: 0.17399, Test Accuracy: 96.30%


In [ ]:
model_print.eval()
with torch.inference_mode():
    y_preds = torch.round(torch.sigmoid(model_print(X_test))).squeeze()
y_preds[:10], y[:10]

(tensor([1., 0., 0., 1., 1., 0., 1., 0., 0., 0.]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [ ]:
y_preds.shape

torch.Size([108])

In [ ]:
y_test.shape

torch.Size([108])

In [ ]:
confusion_matrix(y_test, y_preds)

array([[63,  1],
       [ 3, 41]])

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_preds).ravel()
(tn, fp, fn, tp)

(63, 1, 3, 41)

In [ ]:
#code to calculate accuracy
sum(y_test == y_preds) / len(y_test)

tensor(0.9630)

In [ ]:
precision = tp/(tp+fp)
print (precision)

0.9761904761904762


In [ ]:
#recall
rec = tp/ (tp+fn)
print (rec)

0.9318181818181818
